In [ ]:
!pip install newspaper3k
!pip install jpype1
!pip install konlpy

     |████████████████████████████████| 215kB 4.8MB/s 
     |████████████████████████████████| 194kB 15.1MB/s 
     |████████████████████████████████| 51kB 6.2MB/s 
     |████████████████████████████████| 7.4MB 14.6MB/s 
  Created wheel for feedparser: filename=feedparser-5.2.1-cp36-none-any.whl size=44940 sha256=53f27161cc28742b0cf55ba1d03c3e3c0b8a56725a091e23df66476ec58e4b4b
  Stored in directory: /root/.cache/pip/wheels/8c/69/b7/f52763c41c5471df57703a0ef718a32a5e81ee35dcf6d4f97f
  Created wheel for jieba3k: filename=jieba3k-0.35.1-cp36-none-any.whl size=7398406 sha256=641663dada57df5909827cdffe37c9311388073143df63a4eec3441cc3866714
  Stored in directory: /root/.cache/pip/wheels/83/15/9c/a3f1f67e7f7181170ad37d32e503c35da20627c013f438ed34
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-cp36-none-any.whl size=3357 sha256=350a812c70e4c602c248ced5b6025829b0c121274b6beb5db2f9bfd7280a59ad
  Stored in directory: /root/.cache/pip/wheels/de/03/ca/778e3a7a627e3d98836cc890e7cb40c757

In [ ]:
from newspaper import Article
from konlpy.tag import Kkma
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
import numpy as np

class SentenceTokenizer(object):
  def __init__(self):
    self.kkma = Kkma()
    self.okt = Okt()
    self.stopwords = [ ]
   
  def get_nouns(self, sentences):
    nouns = []
    for sentence in sentences:
      print(sentence)
      if sentence is not '':
        print([noun for noun in self.okt.nouns(str(sentence))
                              if noun not in self.stopwords and len(noun) > 1])
        nouns.append(' '.join([noun for noun in self.okt.nouns(str(sentence))
                              if noun not in self.stopwords and len(noun) > 1]))
    return nouns

class GraphMatrix(object):
  def __init__(self):
    self.tfidf = TfidfVectorizer()
    self.cnt_vec = CountVectorizer()
    self.graph_sentence = []

  def build_sent_graph(self, sentence):
    tfidf_mat = self.tfidf.fit_transform(sentence).toarray()
    self.graph_sentence = np.dot(tfidf_mat, tfidf_mat.T)
    return self.graph_sentence
    
  def build_words_graph(self, sentence):
    cnt_vec_mat = normalize(self.cnt_vec.fit_transform(sentence).toarray().astype(float), axis=0)
    vocab = self.cnt_vec.vocabulary_
    return np.dot(cnt_vec_mat.T, cnt_vec_mat), {vocab[word] : word for word in vocab}

class Rank(object):
  def get_ranks(self, graph, d=0.85): # d = damping factor
    A = graph
    matrix_size = A.shape[0]
    for id in range(matrix_size):
      A[id, id] = 0 # diagonal 부분을 0으로
      link_sum = np.sum(A[:,id]) # A[:, id] = A[:][id]
      if link_sum != 0:
        A[:, id] /= link_sum
      A[:, id] *= -d
      A[id, id] = 1

    B = (1-d) * np.ones((matrix_size, 1))
    ranks = np.linalg.solve(A, B) # 연립방정식 Ax = b
    return {idx: r[0] for idx, r in enumerate(ranks)}

class TextRank(object):
  def __init__(self, text):

      self.sent_tokenize = SentenceTokenizer()
      self.sentences = text
      self.nouns = self.sent_tokenize.get_nouns(self.sentences)
      print(self.nouns)
      self.graph_matrix = GraphMatrix()
      self.sent_graph = self.graph_matrix.build_sent_graph(self.nouns)
      self.words_graph, self.idx2word = self.graph_matrix.build_words_graph(self.nouns)
      self.rank = Rank()
      self.sent_rank_idx = self.rank.get_ranks(self.sent_graph)
      self.sorted_sent_rank_idx = sorted(self.sent_rank_idx, key=lambda k: self.sent_rank_idx[k], reverse=True)
      self.word_rank_idx = self.rank.get_ranks(self.words_graph)
      self.sorted_word_rank_idx = sorted(self.word_rank_idx, key=lambda k: self.word_rank_idx[k], reverse=True)
      
  def summarize(self, sent_num=3):
    summary = []
    index=[]
    for idx in self.sorted_sent_rank_idx[:sent_num]:
      index.append(idx)
      index.sort()
    for idx in index:
      summary.append(self.sentences[idx])
    
    return summary
  
  def keywords(self, word_num=10):
    rank = Rank()
    rank_idx = rank.get_ranks(self.words_graph)
    sorted_rank_idx = sorted(rank_idx, key=lambda k: rank_idx[k], reverse=True)
    keywords = []
    index=[]
    for idx in sorted_rank_idx[:word_num]:
      index.append(idx)
    for idx in index:
      keywords.append(self.idx2word[idx])
    return keywords

sents = [
  '경찰 관계자들이 19일 오후 서울 강북구 오패산 터널 인근에서 사제 총기를 발사해 경찰을 살해한 용의자 성모씨를 검거하고 있다. 성씨는 검거 당시 서바이벌 게임에서 쓰는 방탄조끼에 헬멧까지 착용한 상태였다.', 
  '사제 총기로 경찰을 살해한 범인 성모 46 씨는 주도면밀했다.', 
  '경찰에 따르면 성씨는 19일 오후 강북경찰서 인근 부동산 업소 밖에서 부동산업자 이모 67 씨가 나오기를 기다렸다. 이씨와는 평소에도 말다툼을 자주 한 것으로 알려졌다.', 
  '이씨가 나와 걷기 시작하자 성씨는 따라가면서 미리 준비해온 사제 총기를 이씨에게 발사했다 총알이 빗나가면서 이씨는 도망갔다. 그 빗나간 총알은 지나가던 행인 71 씨의 배를 스쳤다.', 
  '성씨는 강북서 인근 치킨집까지 이씨 뒤를 쫓으며 실랑이하다 쓰러뜨린 후 총기와 함께 가져온 망치로 이씨 머리를 때렸다.', 
  '이 과정에서 오후 6시 20분께 강북구 번동 길 위에서 사람들이 싸우고 있다 총소리가 났다 는 등의 신고가 여러건 들어왔다.', 
  '5분 후에 성씨의 전자발찌가 훼손됐다는 신고가 보호관찰소 시스템을 통해 들어왔다. 성범죄자로 전자발찌를 차고 있던 성씨는 부엌칼로 직접 자신의 발찌를 끊었다.', 
  '서울 시내에서 폭행 용의자가 현장 조사를 벌이던 경찰관에게 사제총기를 발사해 경찰관이 숨졌다. 19일 오후 6시28분 강북구 번동에서 둔기로 맞았다는 폭행 피해 신고가 접수돼 현장에서 조사하던 강북경찰서 번동파출소 소속 김모 54 경위가 폭행 용의자 성모 45 씨가 쏜 사제총기에 맞고 쓰러진 뒤 병원에 옮겨졌으나 숨졌다.', 
  '신고를 받고 번동파출소에서 김창호 54 경위 등 경찰들이 오후 6시 29분께 현장으로 출동했다. 성씨는 그사이 부동산 앞에 놓아뒀던 가방을 챙겨 오패산 쪽으로 도망간 후였다.', 
  '김 경위는 오패산 터널 입구 오른쪽의 급경사에서 성씨에게 접근하다가 오후 6시 33분께 풀숲에 숨은 성씨가 허공에 난사한 10여발의 총알 중 일부를 왼쪽 어깨 뒷부분에 맞고 쓰러졌다.', 
  '김 경위는 구급차가 도착했을 때 이미 의식이 없었고 심폐소생술을 하며 병원으로 옮겨졌으나 총알이 폐를 훼손해 오후 7시 40분께 사망했다.', 
  '김 경위는 외근용 조끼를 입고 있었으나 총알을 막기에는 역부족이었다.', 
  '머리에 부상을 입은 이씨도 함께 병원으로 이송됐으나 생명에는 지장이 없는 것으로 알려졌다.', 
  '성씨는 오패산 터널 밑쪽 숲에서 오후 6시 45분께 잡혔다.', 
  '19일 오후 서울 강북구 오패산 터널 인근에서 경찰들이 폭행 용의자가 사제총기를 발사해 경찰관이 사망한 사건을 조사 하고 있다.', 
  '총 때문에 쫓던 경관들과 민간인들이 몸을 숨겼는데 인근 신발가게 직원 이모씨가 다가가 성씨를 덮쳤고 이어 현장에 있던 다른 상인들과 경찰이 가세해 체포했다.', 
  '성씨는 경찰에 붙잡힌 직후 나 자살하려고 한 거다 맞아 죽어도 괜찮다고 말한 것으로 전해졌다.', 
  '성씨 자신도 경찰이 발사한 공포탄 1발 실탄 3발 중 실탄 1발을 배에 맞았으나 방탄조끼를 입은 상태여서 부상하지는 않았다.', 
  '경찰은 인근을 수색해 성씨가 만든 사제총 16정과 칼 7개를 압수했다. 실제 폭발할지는 알 수 없는 요구르트병에 무언가를 채워두고 심지를 꽂은 사제 폭탄도 발견됐다.', 
  '일부는 숲에서 발견됐고 일부는 성씨가 소지한 가방 안에 있었다.'
]

textrank = TextRank(sents)
for row in textrank.summarize(3):
  print(row)
  print()
print('keywords :',textrank.keywords())

경찰 관계자들이 19일 오후 서울 강북구 오패산 터널 인근에서 사제 총기를 발사해 경찰을 살해한 용의자 성모씨를 검거하고 있다. 성씨는 검거 당시 서바이벌 게임에서 쓰는 방탄조끼에 헬멧까지 착용한 상태였다.
['경찰', '관계자', '오후', '서울', '강북구', '오패산', '터널', '인근', '사제', '총기', '발사', '경찰', '살해', '용의자', '성모', '검거', '성씨', '검거', '당시', '서바이벌', '게임', '방탄', '조끼', '헬멧', '착용', '상태']
사제 총기로 경찰을 살해한 범인 성모 46 씨는 주도면밀했다.
['사제', '총기', '경찰', '살해', '범인', '성모', '주도']
경찰에 따르면 성씨는 19일 오후 강북경찰서 인근 부동산 업소 밖에서 부동산업자 이모 67 씨가 나오기를 기다렸다. 이씨와는 평소에도 말다툼을 자주 한 것으로 알려졌다.
['경찰', '성씨', '오후', '강북', '경찰서', '인근', '부동산', '부동산', '업자', '이모', '이씨', '평소', '말다툼', '자주']
이씨가 나와 걷기 시작하자 성씨는 따라가면서 미리 준비해온 사제 총기를 이씨에게 발사했다 총알이 빗나가면서 이씨는 도망갔다. 그 빗나간 총알은 지나가던 행인 71 씨의 배를 스쳤다.
['이씨', '걷기', '시작', '성씨', '미리', '준비', '사제', '총기', '이씨', '발사', '총알', '이씨', '총알', '행인']
성씨는 강북서 인근 치킨집까지 이씨 뒤를 쫓으며 실랑이하다 쓰러뜨린 후 총기와 함께 가져온 망치로 이씨 머리를 때렸다.
['성씨', '강북', '인근', '치킨', '이씨', '실랑이', '총기', '망치', '이씨', '머리']
이 과정에서 오후 6시 20분께 강북구 번동 길 위에서 사람들이 싸우고 있다 총소리가 났다 는 등의 신고가 여러건 들어왔다.
['과정', '오후', '강북구', '번동', '사람', '소리', '신고']
5분 후에 성씨의 전자발찌가 훼손됐다는 신고가 보호관